<a href="https://colab.research.google.com/github/joblazek/psp-auction/blob/main/PSP_ladder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:


from __future__ import annotations

import heapq
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List, Optional

# ------------------------------
# These functions are defined in util.py, we may assume they are available
# ------------------------------
"""
def setup_experiment(experiment: str | None = None) -> Dict:
def _alloc_greedy_j(M: Dict, j: int) -> np.ndarray:
def _update_p_star_j(M: Dict, j: int):
def market_snapshot(M: Dict) -> pd.DataFrame:
def market_totals(M: Dict) -> Tuple[float, float, float]:
def market_totals_by_seller(M: Dict) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
def market_revenue_by_seller(M: Dict) -> np.ndarray:
def per_seller_totals(M: Dict) -> pd.DataFrame:
def plot_buyer_diagnostics(M: Dict, i: int, *, show_jbr: bool = True, num_points: int = 200):
def plot_utility_surface(M: Dict, i: int, j: int, q_steps: int = 50, p_steps: int = 50):
def plot_connectivity(M: Dict, *, title: Optional[str] = None, show_labels: bool = True):
def randomize_bids(M: Dict):
def clustered_membership(I: int, J: int) -> np.array:
def random_bipartite(I:int, J: int, density: float) -> np.array:
"""
# ------------------------------
# Event types
# ------------------------------
BUYER_COMPUTE = 1
POST_BID = 2

# ------------------------------
# Market construction
# ------------------------------

def make_market_multi(I: int,
                      J: int,
                      Q_max: float | np.ndarray = 100.0,
                      epsilon: float = 5.0,
                      reserve: float | np.ndarray = 0.0,
                      budget_range=(5.0, 20.0),
                      q_range=(10.0, 60.0),
                      kappa_range=(1.0, 3.5),
                      seed: int = 12345,
                      jitter: float = 0.01,
                      adj: Optional[np.ndarray] = None) -> Dict:
    """Create a multi-seller PSP market state as flat arrays.

    Buyer arrays (len I): b, qbar, kappa, gen
    Seller arrays (len J): Q_max, reserve, p_star
    Bid arrays (I×J): bid_q, bid_p
    """
    rng = np.random.default_rng(seed)
    b = rng.uniform(*budget_range, size=I)
    qbar = rng.uniform(*q_range, size=I)
    kappa = rng.uniform(*kappa_range, size=I)

    Q_max = np.full(J, float(Q_max)) if np.isscalar(Q_max) else np.asarray(Q_max, dtype=float)
    assert Q_max.shape == (J,)
    reserve = np.full(J, float(reserve)) if np.isscalar(reserve) else np.asarray(reserve, dtype=float)
    assert reserve.shape == (J,)

    bid_q = np.zeros((I, J), dtype=float)
    bid_p = np.zeros((I, J), dtype=float)
    p_star = np.array(reserve, dtype=float)

    M = {
        "I": I,
        "J": J,
        "b": np.ascontiguousarray(b, dtype=float),
        "qbar": np.ascontiguousarray(qbar, dtype=float),
        "kappa": np.ascontiguousarray(kappa, dtype=float),
        "Q_max": np.ascontiguousarray(Q_max, dtype=float),
        "reserve": np.ascontiguousarray(reserve, dtype=float),
        "bid_q": np.ascontiguousarray(bid_q, dtype=float),
        "bid_p": np.ascontiguousarray(bid_p, dtype=float),
        "p_star": np.ascontiguousarray(p_star, dtype=float),
        "adj": np.ascontiguousarray(adj, dtype=bool),
        "epsilon": float(epsilon),
        # async engine
        "pq": [],
        "seq": 0,
        "t": 0.0,
        "jitter": float(jitter),
        "rng": rng,
        # buyer generations (staleness guard for POST_BID)
        "gen": np.zeros(I, dtype=np.int64),
        # JBR grid resolution per-buyer (can be customized)
        "grid_K": 64,
        # tolerances
        "tol": 1e-12,
        # convergence bookkeeping
        "pending_posts": 0,
        "events_since_apply": 0,
        "chaos": False,
        # Lamport clock
        "lc_buyer": np.zeros(I, dtype=np.int64),
        "lc_cell": np.zeros((I, J), dtype=np.int64),
        "retro_applies": 0,
        "retro_mass": 0,
        "retro_hist": {},
        # policy knobs:
        "endpoint_policy": "qjc",           # 'lower' | 'upper' | 'qjc'
        "tie_policy": "qjc_qty",            # 'open' | 'qjc_count' | 'qjc_qty'
        "split_rule": "subset_uniform",     # 'subset_uniform' | 'min_cost'
        "budget_policy": "reject",          # 'reject' | 'scale_uniform' | 'scale_consistent'
    }
    return M

# ------------------------------
# Buyer valuation and derivatives
# ------------------------------

def theta_i(i: int, z: float, M: Dict) -> float:
    """θ_i(z) = κ_i * qbar_i * m − 0.5 * κ_i * m^2, with m = min(z, qbar_i)."""
    m = min(z, float(M["qbar"][i]))
    k = float(M["kappa"][i])
    return k * float(M["qbar"][i]) * m - 0.5 * k * m * m


def theta_i_prime(i: int, z: float, M: Dict) -> float:
    """θ'_i(z) = κ_i * (qbar_i − z) for z ≤ qbar_i; 0 beyond."""
    q = float(M["qbar"][i])
    k = float(M["kappa"][i])
    return k * (q - z) if z < q else 0.0

# ------------------------------
# Seller-side primitives (columnwise)
# ------------------------------

def _others_mask(i: int, I: int) -> np.ndarray:
    m = np.ones(I, dtype=bool); m[i] = False; return m


def Q_i_j(i: int, j: int, p: float, M: Dict) -> float:
    """Q_i^j(p): remaining capacity at seller j after removing opponents with price > p."""
    mask = _others_mask(i, M["I"]) ; qcol = M["bid_q"][mask, j] ; pcol = M["bid_p"][mask, j]
    rem = M["Q_max"][j] - float(np.sum(qcol[pcol > p]))
    return rem if rem > 0.0 else 0.0


def Qbar_i_j(i: int, j: int, p: float, M: Dict) -> float:
    """Q̄_i^j(p): remaining capacity at seller j after removing opponents with price ≥ p."""
    mask = _others_mask(i, M["I"]) ; qcol = M["bid_q"][mask, j] ; pcol = M["bid_p"][mask, j]
    rem = M["Q_max"][j] - float(np.sum(qcol[pcol >= p]))
    return rem if rem > 0.0 else 0.0


def P_i_j(i: int, j: int, z: float, M: Dict) -> float:
    """Price density P_i^j(z) = inf{ y ≥ 0 : Q_i^j(y) ≥ z }. Evaluate across candidate prices."""
    others = _others_mask(i, M["I"]) ; cand = np.unique(np.concatenate(([0.0], M["bid_p"][others, j])))
    for y in cand:
        if Q_i_j(i, j, float(y), M) >= z:
            return float(y)
    return float("inf")


def integral_P_i_j(i: int, j: int, a: float, M: Dict, N: int = 100) -> float:
    if a <= 0.0: return 0.0
    zs = np.linspace(0.0, a, N + 1)
    Ps = np.array([P_i_j(i, j, float(zk), M) for zk in zs])
    dz = a / N
    return float(np.trapezoid(Ps, dx=dz))

# ------------------------------
# Current allocation/cost/utility for buyer i (snapshot based)
# ------------------------------

def a_row(i: int, M: Dict) -> Tuple[np.ndarray, float]:
    """Return per-seller allocations a_ij for current bids, and their total."""
    J = M["J"]
    a = np.zeros(J, dtype=float)
    for j in range(J):
        q, p = float(M["bid_q"][i, j]), float(M["bid_p"][i, j])
        a[j] = min(q, Qbar_i_j(i, j, p, M))
    return a, float(np.sum(a))

def cost_row(i: int, a: np.ndarray, M: Dict) -> float:
    J = M["J"]; return float(sum(integral_P_i_j(i, j, float(a[j]), M) for j in range(J)))

def u_i_current(i: int, M: Dict) -> float:
    a, atot = a_row(i, M)
    cost = cost_row(i, a, M)
    return theta_i(i, atot, M) - cost

def build_ladders(i: int, M: Dict) -> Dict:
    """
    Build per-seller price ladders of opponents for buyer i:
      - per_seller[j]: dict with fields
          'p'    : opponents' posted prices at seller j (ascending, filtered by adjacency)
          'q'    : aligned opponents' quantities
          'pref' : prefix sums of q (pref[t] = sum_{<t} q_sorted)
          'suf'  : suffix sums of q (suf[t]  = sum_{>=t} q_sorted)
      - steps: sorted global union of {0} ∪ {opponent prices across all sellers} ∪ {w_max}
      - w_max: θ'_i(0)
    """
    I, J = int(M["I"]), int(M["J"])
    others = np.ones(I, dtype=bool); others[i] = False
    has_adj = M.get("adj") is not None

    per_seller = []
    all_steps = [0.0]
    for j in range(J):
        if has_adj:
            opp_ok = M["adj"][others, j]
        else:
            opp_ok = np.ones(I - 1, dtype=bool)
        pcol = np.asarray(M["bid_p"][others, j][opp_ok], dtype=float)
        qcol = np.asarray(M["bid_q"][others, j][opp_ok], dtype=float)
        if pcol.size:
            idx = np.argsort(pcol, kind="mergesort")
            p_sorted = pcol[idx]
            q_sorted = qcol[idx]
            n = p_sorted.size
            pref = np.empty(n + 1, dtype=float); pref[0] = 0.0
            for t in range(n): pref[t + 1] = pref[t] + float(q_sorted[t])
            suf  = np.empty(n + 1, dtype=float); suf[n] = 0.0
            for t in range(n - 1, -1, -1): suf[t] = suf[t + 1] + float(q_sorted[t])
            per_seller.append({"p": p_sorted, "q": q_sorted, "pref": pref, "suf": suf})
            all_steps.extend(np.unique(p_sorted).tolist())
        else:
            per_seller.append({"p": np.array([], float),
                               "q": np.array([], float),
                               "pref": np.array([0.0], float),
                               "suf":  np.array([0.0], float)})
    w_max = float(theta_i_prime(i, 0.0, M))
    steps = np.unique(np.concatenate([np.asarray(all_steps, float), np.array([w_max], float)]))
    return {"per_seller": per_seller, "steps": steps, "w_max": w_max}

def _rem_gt_at(i: int, j: int, y: float, M: Dict, ladders: Dict) -> float:
    """
    Remaining capacity at seller j after removing opponents with price > y (strict '>').
    O(log I) using the ladder. Respects adjacency (i,j).
    """
    if M.get("adj") is not None and not bool(M["adj"][i, j]):
        return 0.0
    L = ladders["per_seller"][j]; p_sorted, suf = L["p"], L["suf"]
    if p_sorted.size == 0:
        return float(M["Q_max"][j])
    idx = int(np.searchsorted(p_sorted, float(y), side="right"))
    taken = float(suf[idx])
    rem = float(M["Q_max"][j]) - taken
    return rem if rem > 0.0 else 0.0


def _equals_mass_at(j: int, y: float, ladders: Dict) -> float:
    """
    Total opponents' quantity at seller j with price exactly y.
    """
    L = ladders["per_seller"][j]; p_sorted, pref = L["p"], L["pref"]
    if p_sorted.size == 0:
        return 0.0
    left  = int(np.searchsorted(p_sorted, float(y), side="left"))
    right = int(np.searchsorted(p_sorted, float(y), side="right"))
    return float(pref[right] - pref[left])


def _equals_count_at(j: int, y: float, ladders: Dict) -> int:
    """
    Number of opponents at seller j with price exactly y (for count-based QJC).
    """
    L = ladders["per_seller"][j]; p_sorted = L["p"]
    if p_sorted.size == 0:
        return 0
    left  = int(np.searchsorted(p_sorted, float(y), side="left"))
    right = int(np.searchsorted(p_sorted, float(y), side="right"))
    return int(max(0, right - left))

def seller_caps_at(i: int, y: float, M: Dict, ladders: Dict, *, tie_policy: str = "open") -> np.ndarray:
    """
    Caps z_caps[j] available to buyer i at price y under a tie policy:
      - 'open'       : strict '>' (open-interval) rule
      - 'qjc_count'  : boundary y, split residual among equals by COUNT (approx QJC)
      - 'qjc_qty'    : boundary y, quantity-proportional QJC (requires post-adjustment; here we return 'open' caps as an upper bound)
    """
    J = int(M["J"]); caps = np.zeros(J, dtype=float)
    for j in range(J):
        rem = _rem_gt_at(i, j, y, M, ladders)
        if tie_policy == "open":
            caps[j] = rem
        elif tie_policy == "qjc_count":
            # Split residual among equal-price group (opponents at y plus buyer i)
            eq_cnt = 1 + _equals_count_at(j, y, ladders)
            caps[j] = (rem / float(eq_cnt)) if rem > 0.0 else 0.0
        elif tie_policy == "qjc_qty":
            # We cannot determine a quantity-proportional cap without q_ij.
            # Return the open cap; we'll enforce QJC proportionally after we pick q_row.
            caps[j] = rem
        else:
            raise ValueError(f"Unknown tie_policy={tie_policy}")
    return caps

def sup_G_i_multi(i: int, M: Dict, ladders: Dict, *,
                  endpoint_policy: str = "qjc",
                  tie_policy: str = "open",
                  split_rule: str = "subset_uniform",
                  budget_policy: str = "reject"
                  ) -> Tuple[float, np.ndarray, float, Dict]:
    """
    Stepwise selection of w*: scan open intervals (y_k, y_{k+1}) for interior fixed point.
    If none, choose a boundary according to endpoint_policy and return caps there.

    endpoint_policy:
      - 'lower' : choose y* = y_k  (left boundary closest to the crossing), caps at y_k with tie_policy
      - 'upper' : choose y* = y_{k+1}^- (≈ y_{k+1}-tol), caps from the open interval (evaluate at y_k, tie='open')
      - 'qjc'   : choose y* = y_k with QJC tie policy at boundary (use tie_policy argument for caps)
      - 'utility_max' : evaluate *both* lower and upper candidates end-to-end
                        (split + budget policy), pick higher utility (tie → lower)

    tie_policy is used for boundary caps at the chosen point ('open' | 'qjc_count' | 'qjc_qty').
    split_rule and budget_policy are only used when endpoint_policy='utility_max'.
    Returns: w_star, caps, z_star, meta  (meta = {'type': 'interior'|'boundary', 'y': y_star})
    """
    tol = float(M["tol"])
    steps = ladders["steps"]; w_max = float(ladders["w_max"])

    # 1) Try to find interior solution
    for k in range(steps.size - 1):
        yk, yk1 = float(steps[k]), float(steps[k + 1])
        if yk1 <= yk + tol:
            continue
        # availability is constant on (yk, yk1): evaluate at left boundary with strict '>'
        z_caps = seller_caps_at(i, yk, M, ladders, tie_policy="open")
        Z = float(np.sum(z_caps))
        w_imp = float(theta_i_prime(i, Z, M))
        if (w_imp > yk + tol) and (w_imp < yk1 - tol):
            z_star = min(float(M["qbar"][i]), Z)
            return w_imp, z_caps, z_star, {"type": "interior", "y": None}

    # 2) No interior: compute f(y_k) = θ'(Z_k) - y_k at each left boundary and locate crossing
    f_vals = []
    for k in range(steps.size - 1):  # exclude last boundary as a left boundary
        yk = float(steps[k])
        z_caps = seller_caps_at(i, yk, M, ladders, tie_policy="open")
        Zk = float(np.sum(z_caps))
        f_vals.append(float(theta_i_prime(i, Zk, M) - yk))

    # If degenerate (no opponents), take y*=0 as "lower"
    if len(f_vals) == 0:
        k_star = 0
    else:
        k_star = max([idx for idx, fv in enumerate(f_vals) if fv >= -tol], default=0)

    y_lower = float(steps[k_star])
    y_upper = float(steps[min(k_star + 1, steps.size - 1)])  # right boundary of that interval

    # Helper to build a *complete* plan at a boundary and return (w, caps, z_star, q_row, p_row, cost, u, feasible, meta)
    def eval_boundary(which: str):
        if which == "lower":
            w = y_lower
            caps = seller_caps_at(i, y_lower, M, ladders, tie_policy=tie_policy)  # allow QJC at lower
            meta = {"type": "boundary", "y": y_lower}
        elif which == "upper":
            # choose w just below the upper boundary; caps from open interval at y_lower
            w = max(y_upper - tol, 0.0)
            caps = seller_caps_at(i, y_lower, M, ladders, tie_policy="open")
            meta = {"type": "boundary", "y": y_upper}
        else:
            raise ValueError(which)

        z_star = min(float(M["qbar"][i]), float(np.sum(caps)))
        q_row, p_row, cost, u_new, feasible = compute_t_i_multi(
            i, M, w, caps, z_star, meta,
            split_rule=split_rule, tie_policy=tie_policy if which == "lower" else "open",
            budget_policy=budget_policy
        )
        return w, caps, z_star, q_row, p_row, cost, u_new, feasible, meta

    ep = str(endpoint_policy).lower()
    if ep != "utility_max":
        # legacy endpoint behaviors
        if ep == "upper":
            w_star = max(y_upper - tol, 0.0)
            caps   = seller_caps_at(i, y_lower, M, ladders, tie_policy="open")
            z_star = min(float(M["qbar"][i]), float(np.sum(caps)))
            return w_star, caps, z_star, {"type": "boundary", "y": y_upper}
        elif ep in ("lower", "qjc"):
            w_star = y_lower
            caps   = seller_caps_at(i, y_lower, M, ladders,
                                    tie_policy=("open" if ep == "lower" else tie_policy))
            z_star = min(float(M["qbar"][i]), float(np.sum(caps)))
            return w_star, caps, z_star, {"type": "boundary", "y": y_lower}
        else:
            # default to lower
            w_star = y_lower
            caps   = seller_caps_at(i, y_lower, M, ladders, tie_policy="open")
            z_star = min(float(M["qbar"][i]), float(np.sum(caps)))
            return w_star, caps, z_star, {"type": "boundary", "y": y_lower}

    # 3) endpoint_policy == 'utility_max': evaluate both boundaries end-to-end and pick higher utility
    wL, capsL, zL, qL, pL, cL, uL, feasL, metaL = eval_boundary("lower")
    wU, capsU, zU, qU, pU, cU, uU, feasU, metaU = eval_boundary("upper")

    # Infeasible candidates get -inf utility
    uL_eff = uL if feasL else -np.inf
    uU_eff = uU if feasU else -np.inf

    # Deterministic tie-break: prefer LOWER if |Δu| ≤ 1e-9
    if (uL_eff > uU_eff + 1e-9) or (abs(uL_eff - uU_eff) <= 1e-9):
        return wL, capsL, zL, metaL
    else:
        return wU, capsU, zU, metaU

def _subset_uniform_split(z_star: float, caps: np.ndarray) -> np.ndarray:
    """
    Choose the smallest subset whose caps sum ≥ z_star (descending caps),
    then equal-split water-fill under per-seller caps.
    """
    J = caps.size
    order = np.argsort(-caps, kind="mergesort")
    cum = 0.0; S: List[int] = []
    for j in order:
        if caps[j] <= 0.0: continue
        S.append(int(j)); cum += float(caps[j])
        if cum >= z_star - 1e-15: break
    q_row = np.zeros(J, dtype=float)
    if z_star <= 0.0 or not S:
        return q_row
    tol_cap = 1e-12
    cap_map = {j: float(caps[j]) for j in S}
    while True:
        active = [j for j in S if q_row[j] < cap_map[j] - tol_cap]
        rem = z_star - float(np.sum(q_row))
        if rem <= tol_cap or not active:
            break
        share = rem / len(active)
        progress = 0.0
        for j in active:
            add = min(share, cap_map[j] - q_row[j])
            if add > 0.0:
                q_row[j] += add; progress += add
        if progress <= tol_cap:
            break
    return q_row

def _min_cost_split(i: int, M: Dict, z_star: float, caps: np.ndarray, ladders: Dict, y_ref: float) -> np.ndarray:
    """
    Utility-max split: choose q_row to minimize PSP cost subject to sum q = z_star and 0 ≤ q ≤ caps.
    Construct per-seller (price, length) segments up to y_ref, merge globally, and take the cheapest units first.
    """
    J = int(M["J"])
    q_row = np.zeros(J, dtype=float)
    if z_star <= 0.0:
        return q_row

    segments: List[Tuple[float, float, int]] = []
    for j in range(J):
        if caps[j] > 1e-15:
            segments.extend(_per_seller_segments_upto(i, j, float(y_ref), M, ladders))

    # Sort by price ascending (stable)
    segments.sort(key=lambda t: t[0])

    rem = float(z_star)
    for price, length, j in segments:
        if rem <= 1e-15:
            break
        room = float(caps[j]) - float(q_row[j])
        if room <= 1e-15:
            continue
        take = min(rem, room, float(length))
        if take > 0.0:
            q_row[j] += take
            rem -= take

    # Numerical clip
    q_row = np.minimum(q_row, caps)
    return q_row

def _per_seller_segments_upto(i: int, j: int, y_ref: float, M: Dict, ladders: Dict) -> List[Tuple[float, float, int]]:
    """
    Return price segments for seller j up to boundary y_ref as a list of
      (price_level, segment_length, j).
    Segment at price y_t represents the additional capacity unlocked exactly at y_t.
    Includes the y=0 segment if any capacity is available at 0.
    Respects adjacency (i,j).
    """
    if M.get("adj") is not None and not bool(M["adj"][i, j]):
        return []

    L = ladders["per_seller"][j]
    p_sorted = L["p"]

    # All local step prices up to y_ref (inclusive), plus 0
    if p_sorted.size:
        steps_j = np.unique(np.concatenate(([0.0], p_sorted[p_sorted <= float(y_ref)])))
    else:
        steps_j = np.array([0.0], dtype=float)

    segs: List[Tuple[float, float, int]] = []
    prev_rem = 0.0
    for y in steps_j:
        rem = _rem_gt_at(i, j, float(y), M, ladders)   # remaining capacity with strict '>'
        delta = rem - prev_rem
        if delta > 1e-15:
            # Units first available at price y get priced at y
            segs.append((float(y), float(delta), int(j)))
        prev_rem = rem
    return segs

def _apply_budget_policy(i: int, M: Dict, q_row: np.ndarray, base_cost: float, *,
                         policy: str = "reject") -> Tuple[np.ndarray, float, float, bool]:
    """
    Returns (q_row_new, cost_new, u_new, feasible).
    policy:
      - 'reject'       : if cost>b_i, reject
      - 'scale_uniform': scale q_row by λ = min(1, b_i/cost)
      - 'scale_consistent':
            bisection on λ∈[0,1], for each λ set Z=λ*Z0, w=θ'(Z),
            recompute caps at the bracketed step, re-split to exactly Z,
            compute cost, continue until cost≤b_i within tolerance.
    """
    b_i = float(M["b"][i])
    Z0 = float(np.sum(q_row))

    def util_from(qv: np.ndarray, c: float) -> float:
        return theta_i(i, float(np.sum(qv)), M) - float(c)

    # trivial case
    if base_cost <= b_i + 1e-12 or Z0 <= 0.0:
        return q_row, base_cost, util_from(q_row, base_cost), True

    policy = policy.lower()
    if policy == "reject":
        return q_row, base_cost, util_from(q_row, base_cost), False

    if policy == "scale_uniform":
        lam = max(0.0, min(1.0, b_i / float(base_cost)))
        q_new = q_row * lam
        c_new = float(base_cost) * lam
        return q_new, c_new, util_from(q_new, c_new), True

    if policy == "scale_consistent":
        ladders = build_ladders(i, M)  # reuse per-iteration; small cost
        steps = ladders["steps"]; tol = float(M["tol"])
        lo, hi = 0.0, 1.0
        q_best, c_best, u_best = np.zeros_like(q_row), 0.0, -np.inf
        for _ in range(40):
            lam = 0.5 * (lo + hi)
            Zt = lam * Z0
            # implied w and caps at the containing step
            w_hat = float(theta_i_prime(i, Zt, M))
            # find interval [yk, yk1] containing w_hat
            k = int(np.searchsorted(steps, w_hat, side="right")) - 1
            k = max(0, min(k, steps.size - 2))
            yk, yk1 = float(steps[k]), float(steps[k + 1])
            boundary = abs(w_hat - yk) <= 1e-12 or abs(w_hat - yk1) <= 1e-12
            caps = seller_caps_at(i, yk, M, ladders, tie_policy="open")
            z_star = min(float(M["qbar"][i]), float(np.sum(caps)), Zt)
            q_tmp = _subset_uniform_split(z_star, caps)
            # (Optional) QJC quantity adjustment at boundary if your tie policy demands it
            # We keep 'open' here for speed/consistency.
            c_tmp = cost_row(i, q_tmp, M)
            if c_tmp <= b_i + 1e-12:
                lo = lam
                if c_tmp > c_best:
                    q_best, c_best, u_best = q_tmp, c_tmp, util_from(q_tmp, c_tmp)
            else:
                hi = lam
            if hi - lo <= 1e-3:
                break
        if c_best <= b_i + 1e-12:
            return q_best, c_best, u_best, True
        # fallback uniform
        lam = max(0.0, min(1.0, b_i / float(base_cost)))
        q_new = q_row * lam
        c_new = float(base_cost) * lam
        return q_new, c_new, util_from(q_new, c_new), True

    raise ValueError(f"Unknown budget_policy={policy}")

def compute_t_i_multi(i: int, M: Dict,
                      w_star: float, caps: np.ndarray, z_star: float, meta: Dict, *,
                      split_rule: str = "subset_uniform",
                      tie_policy: str = "open",
                      budget_policy: str = "reject") -> Tuple[np.ndarray, np.ndarray, float, float, bool]:
    """
    Build (q_row, p_row) from (w_star, caps, z_star):
      - split_rule: 'subset_uniform' | 'min_cost'
      - tie_policy: if 'qjc_qty' and meta['type']=='boundary', apply proportional adjustment
      - budget_policy: 'reject'|'scale_uniform'|'scale_consistent'
    """
    # 1) split to z_star under caps
    if split_rule == "subset_uniform":
        q_row = _subset_uniform_split(z_star, caps)

    elif split_rule == "min_cost":
        # derive the reference boundary y_ref that defines the caps we computed
        ladders = build_ladders(i, M)
        steps = ladders["steps"]
        # Use the left boundary of the interval containing w_star
        k = int(np.searchsorted(steps, float(w_star), side="right")) - 1
        k = max(0, min(k, steps.size - 2))
        y_ref = float(steps[k])
        q_row = _min_cost_split(i, M, z_star, caps, ladders, y_ref)

    else:
        raise ValueError(f"split_rule={split_rule} not implemented")

    # 2) boundary QJC (quantity) post-adjustment if requested
    if tie_policy == "qjc_qty" and meta.get("type") == "boundary":
        y_star = float(meta.get("y", w_star))
        J = int(M["J"])
        ladders = build_ladders(i, M)
        for j in range(J):
            if q_row[j] <= 0.0:
                continue
            r = _rem_gt_at(i, j, y_star, M, ladders)
            E = _equals_mass_at(j, y_star, ladders)
            denom = q_row[j] + E
            if denom > 0.0:
                q_share = r * (q_row[j] / denom)
                if q_share < q_row[j]:
                    q_row[j] = q_share

    # 3) prices
    p_row = np.full(int(M["J"]), float(w_star), dtype=float)

    # 4) cost & budget
    base_cost = cost_row(i, q_row, M)
    q_row, cost_new, u_new, feasible = _apply_budget_policy(
        i, M, q_row, base_cost, policy=budget_policy
    )
    return q_row, p_row, cost_new, u_new, feasible

def joint_best_response_plan(i: int, M: Dict) -> Tuple[np.ndarray, np.ndarray, bool, float]:
    # read policies (with defaults)
    endpoint_policy = str(M.get("endpoint_policy", "qjc")).lower()
    tie_policy      = str(M.get("tie_policy", "open")).lower()
    split_rule      = str(M.get("split_rule", "subset_uniform")).lower()
    budget_policy   = str(M.get("budget_policy", "reject")).lower()

    ladders = build_ladders(i, M)
    w_star, caps, z_star, meta = sup_G_i_multi(
        i, M, ladders,
        endpoint_policy=endpoint_policy,
        tie_policy=("open" if endpoint_policy == "lower" else tie_policy),
        split_rule=split_rule,
        budget_policy=budget_policy
    )

    q_row, p_row, cost, u_new, feasible = compute_t_i_multi(
        i, M, w_star, caps, z_star, meta,
        split_rule=split_rule, tie_policy=tie_policy, budget_policy=budget_policy
    )
    return q_row, p_row, feasible, u_new

# ------------------------------
# Priority queue helpers
# ------------------------------

def push(M: Dict, t: float, etype: int, payload: Tuple):
    M["seq"] += 1
    heapq.heappush(M["pq"], (float(t), int(M["seq"]), int(etype), payload))


def pop(M: Dict):
    return heapq.heappop(M["pq"]) if M["pq"] else None


def schedule_all_buyers(M: Dict, t0: float = 0.0):
    rng = M["rng"]
    for i in range(M["I"]):
        push(M, t0 + float(rng.random() * M["jitter"]), BUYER_COMPUTE, (i,))

# ------------------------------
# Event handlers
# ------------------------------

def handle_buyer_compute(M: Dict, i: int, verbose: bool = False, debug: bool = False):
    base_t = M["t"]
    ensure_lamport(M)

    # Buyer i "reads" the market and ticks its Lamport clock.
    # If you prefer a stricter read, use row max: seen = int(np.max(M["lc_cell"][i, :]))
    seen = int(np.max(M["lc_cell"]))  # global seen; ok for our simulation
    M["lc_buyer"][i] = int(max(int(M["lc_buyer"][i]), seen) + 1)

    old_u = u_i_current(i, M)
    q_row, p_row, feasible, u_new = joint_best_response_plan(i, M)

    # Always re-enter the queue (asynchronous cadence)
    push(M, base_t + 1.0 + float(M["rng"].random() * M["jitter"]), BUYER_COMPUTE, (i,))

    if (not feasible) or (u_new <= old_u + float(M["epsilon"])):
        if verbose:
            print(f"BUYER_COMPUTE i={i}: no accepted update (feasible={feasible}, Δu={u_new-old_u:.6f})")
        return False

    # Accept: schedule one POST per changed/active cell, carrying buyer's Lamport tag
    J = int(M["J"]); tol = 1e-12
    changed = False
    num_posts = 0
    lc_send = int(M["lc_buyer"][i])

    for j in range(J):
        if M.get("adj") is not None and not M["adj"][i, j]:
            continue
        dq = abs(float(M["bid_q"][i, j]) - float(q_row[j]))
        dp = abs(float(M["bid_p"][i, j]) - float(p_row[j]))
        old_q = float(M["bid_q"][i, j]); new_q = float(q_row[j])
        # post either if numerically different OR one of them is nonzero (activates/clears)
        if dq > tol or ((old_q > 0.0) or (new_q > 0.0)) or dp > tol:
            changed = True
            num_posts += 1
            t_post = base_t + float(M["rng"].random() * M["jitter"])
            # payload no longer includes 'gen'—only lc (buyer-side guard)
            push(M, t_post, POST_BID, (i, j, float(q_row[j]), float(p_row[j]), int(lc_send)))

    if num_posts:
        M["pending_posts"] = int(M.get("pending_posts", 0)) + num_posts

    if verbose:
        print(f"BUYER_COMPUTE i={i}: {'accepted' if changed else 'no-op'} plan, posts scheduled: {num_posts}")
    return changed

def handle_post_bid(M: Dict, i: int, j: int, q: float, p: float, lc_send: int,
                    verbose: bool = False, debug: bool = False):
    ensure_lamport(M)

    # Block if (i,j) is not connected
    if M.get("adj") is not None and not M["adj"][i, j]:
        if M.get("pending_posts", 0) > 0:
            M["pending_posts"] -= 1
        if verbose:
            print(f"POST_BID blocked (no edge): i={i}, j={j}")
        return False

    clock = str(M.get("clock", "generational")).lower()
    lc_prev = int(M["lc_cell"][i, j])

    if clock == "generational":
        # Pure buyer-side guard (compare-and-swap on buyer's cell Lamport tag)
        if int(lc_send) < lc_prev:
            # stale (arrived older than the last applied to this cell)
            M["stale_drops"] = int(M.get("stale_drops", 0)) + 1
            if M.get("pending_posts", 0) > 0:
                M["pending_posts"] -= 1
            if debug:
                print(f"POST_BID stale drop (buyer-side): i={i}, j={j}, lc_send={lc_send} < lc_cell={lc_prev}")
            return False

        # fresh enough → apply
        q_old = float(M["bid_q"][i, j]); p_old = float(M["bid_p"][i, j])
        M["bid_q"][i, j] = float(q); M["bid_p"][i, j] = float(p)
        M["lc_cell"][i, j] = int(max(lc_prev, lc_send))  # CAS commit
        _update_p_star_j(M, j)

        if M.get("pending_posts", 0) > 0:
            M["pending_posts"] -= 1
        M["events_since_apply"] = 0

        if verbose:
            print(f"POST_BID applied: i={i}, j={j}, (q,p)=({q_old:.4f},{p_old:.4f}) -> ({q:.4f},{p:.4f}), "
                  f"p_star[{j}]={M['p_star'][j]:.4f}")
        return True

    else:
        # chaos mode: accept regardless; record retro metrics
        retro = max(0, lc_prev - int(lc_send))
        if retro > 0:
            M["retro_applies"] = int(M.get("retro_applies", 0)) + 1
            M["retro_mass"] = int(M.get("retro_mass", 0)) + retro
            hist = M.get("retro_hist", {})
            hist[retro] = hist.get(retro, 0) + 1
            M["retro_hist"] = hist

        q_old = float(M["bid_q"][i, j]); p_old = float(M["bid_p"][i, j])
        M["bid_q"][i, j] = float(q); M["bid_p"][i, j] = float(p)
        # advance the cell's Lamport tag to reflect the “latest known” logical time
        M["lc_cell"][i, j] = int(max(lc_prev, int(lc_send)))
        _update_p_star_j(M, j)

        if M.get("pending_posts", 0) > 0:
            M["pending_posts"] -= 1
        M["events_since_apply"] = 0

        if verbose:
            print(f"POST_BID applied (chaos): i={i}, j={j}, (q,p)=({q_old:.4f},{p_old:.4f}) -> ({q:.4f},{p:.4f}), "
                  f"retro={retro}, p_star[{j}]={M['p_star'][j]:.4f}")
        return True

# ------------------------------
# Engine run loop
# ------------------------------

def run(M: Dict, steps: int = 1000, verbose: bool = False, *, idle_event_limit: int | None = None):
    """Run the async engine.

    break_on_convergence: stop when no POST_BID has applied for
      idle_event_limit events and there are no pending POST_BIDs.
    idle_event_limit: default = 4*I (set at runtime if None).
    """
    if idle_event_limit is None:
        idle_event_limit = int(M["I"]) if M["I"] else 4

    for _ in range(steps):
        item = pop(M)
        if item is None:
            break
        t, _, etype, payload = item
        M["t"] = float(t)
        if etype == BUYER_COMPUTE:
            (i,) = payload
            handle_buyer_compute(M, int(i), verbose=verbose)
            # no apply occurred here; count toward idle if nothing applied elsewhere
            M["events_since_apply"] += 1
        elif etype == POST_BID:
            i, j, q, p, lc_send = payload
            handle_post_bid(M, int(i), int(j), float(q), float(p), int(lc_send), verbose=verbose)
            # events_since_apply reset inside handler on apply
        else:
            raise RuntimeError(f"Unknown event type {etype}")

        # Convergence check: no pending posts and long idle since last apply
        if M.get("pending_posts", 0) == 0 and M.get("events_since_apply", 0) >= idle_event_limit:
            print(f"Converged: no price updates for {M['events_since_apply']} events; pending_posts=0")
            break

# ------------------------------
# Demo (__main__)
# ------------------------------
if __name__ == "__main__":
    test = "I=8,J=3,Q=[60, 40, 50],\
              epsilon=2.5,\
              seed=20250823,\
              adj=random:0.5,\
              clock=generational,\
              endpoint=qjc,\
              tie=qjc_qty,\
              split=subset_uniform,\
              budget=reject"
    M = setup_experiment(test)
    randomize_bids(M)
    #print(M["experiment_str"])
    seller_df, buyer_df, meta = print_market_config(M, round_decimals=3, return_frames=True)

    # Schedule and run
    schedule_all_buyers(M, t0=0.0)
    run(M, steps=1000, verbose=True)
    summary = print_snapshot_wide(M,
                                  cols=("Buyer","q_i","p_i","a_i","c_ij"),
                                  round_decimals=3,
                                  include_totals=True)

    summary = print_snapshot_summary(M, round_decimals=2)
    per_seller_df = per_seller_totals(M)
    print("\nPer-seller totals:")
    print(per_seller_df)
    tot = market_totals(M)
    print(f"\nMarket Totals -> TotalAlloc: {tot[0]:.2f}, TotalValue: {tot[1]:.2f}, TotalUtility: {tot[2]:.2f}")

    if False:
        plot_connectivity(M, title="Market connectivity", show_labels=True)
        plot_utility_surface(M, 0, 0)
        plot_buyer_diagnostics(M, i=0, show_jbr=True)



=== Market Configuration ===
I=8, J=3, Q_tot=150.000, Qbar_tot=315.270, adj_density=0.375
epsilon=2.5, jitter=0.01, grid_K=64, tol=1e-12
policies: endpoint=qjc, tie=qjc_qty, split=subset_uniform, budget=reject, chaos=False
runtime: t=0.000, seed=20250823
market totals: alloc=90.004, value=4576.550, cost=1253.444, utility=3323.106

-- Sellers --
 Seller  Q_max  reserve  p_star  posted_q_sum  alloc_sum  demand_ratio  winners  losers  undersubscribed
      0   60.0      0.0   0.000         0.004      0.004         0.000        1       0             True
      1   40.0      0.0  33.628        80.941     40.000         2.024        2       1            False
      2   50.0      0.0  16.225        57.656     50.000         1.153        3       0            False

-- Buyers --
qbar stats (min/mean/max): 16.386/39.409/58.469
kappa stats (min/mean/max): 1.074/2.239/3.172
budget stats (min/mean/max): 6.106/12.659/19.524
 Buyer   qbar  kappa  budget  deg  posted_q_sum  posted_p_max
     0 49.517

/tmp/ipython-input-3872977622.py:915: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  losers = df.groupby("Seller").apply(


In [46]:
# util.py
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

pd.set_option('display.max_colwidth', None)

# ------------------------------
# Experiment string
# ------------------------------

def _fmt_float(x: float) -> str:
    # compact, stable formatting
    if abs(x - round(x)) < 1e-12:
        return str(int(round(x)))
    s = f"{x:.6g}"   # up to ~6 sig figs, trims trailing zeros
    return s

def _fmt_list(arr: np.ndarray) -> str:
    vals = ", ".join(_fmt_float(float(v)) for v in np.asarray(arr).ravel())
    return f"[{vals}]"

def experiment_to_string(M: Dict) -> str:
    """
    Produce a canonical, reproducible experiment string from the market M.
    Prefers M['exp_meta'] if present; otherwise infers from M.
    """
    meta = dict(M.get("exp_meta", {}))  # may be empty; we infer below

    I = int(M["I"]); J = int(M["J"])
    Q = np.asarray(M["Q_max"], dtype=float)
    reserve = np.asarray(M.get("reserve", np.zeros(J)), dtype=float)
    epsilon = float(M.get("epsilon", 0.0))
    seed = int(meta.get("seed", M.get("seed", 0)))  # store seed in meta if you care to

    # Policies (infer from M if not in meta)
    endpoint = str(meta.get("endpoint_policy", M.get("endpoint_policy", "qjc"))).lower()
    tie      = str(meta.get("tie_policy",      M.get("tie_policy",      "open"))).lower()
    split    = str(meta.get("split_rule",      M.get("split_rule",      "subset_uniform"))).lower()
    budget   = str(meta.get("budget_policy",   M.get("budget_policy",   "reject"))).lower()

    # Clock / chaos
    chaos = bool(meta.get("chaos", M.get("chaos", False)))
    clock_str = "chaos" if chaos else "generational"

    # Adjacency spec: prefer the original mode/density if present
    adj = M.get("adj", None)
    adj_mode = meta.get("adj_mode", None)
    density  = meta.get("density", None)
    if adj_mode is None:
        # try to infer
        if adj is None:
            adj_mode = "full"
        else:
            ones = float(np.mean(adj)) if adj.size > 0 else 1.0
            if abs(ones - 1.0) < 1e-12:
                adj_mode = "full"
            else:
                adj_mode = "random"
                density = float(ones)
    if density is None and adj is not None and adj_mode == "random":
        density = float(np.mean(adj))

    # Assemble ordered tokens
    tokens = []
    tokens.append(f"I={I}")
    tokens.append(f"J={J}")
    tokens.append(f"Q={_fmt_list(Q)}")
    if np.any(reserve != 0.0):
        tokens.append(f"\nreserve={_fmt_list(reserve)}\n")
    tokens.append(f"\nepsilon={_fmt_float(epsilon)}")
    if seed:
        tokens.append(f"\nseed={seed}")

    # Adjacency token
    if adj_mode == "full":
        tokens.append("\nadj=full")
    elif adj_mode in ("clustered", "cluster", "clusters"):
        tokens.append("\nadj=clustered")
    else:
        # random with density
        rho = 0.5 if density is None else density
        tokens.append(f"\nadj=random:{_fmt_float(rho)}")

    # Clock & policies
    tokens.append(f"\nclock={clock_str}")
    tokens.append(f"\nendpoint={endpoint}")
    tokens.append(f"\ntie={tie}")
    tokens.append(f"\nsplit={split}")
    tokens.append(f"\nbudget={budget}")

    return ",".join(tokens)

# ------------------------------
# setup_experiment
# ------------------------------

def setup_experiment(experiment: str | None = None) -> Dict:
    """
    Build and configure a market from a compact experiment string.

    Syntax (comma-separated tokens; order doesn't matter):
      Sizes / capacities:
        I=8, J=3
        Q=[60,40,50]          # per-seller capacities (sets J=3)
        Q=100                 # scalar capacity for all sellers
        reserve=[0,0,0]       # per-seller reserve (default zeros)
        epsilon=2.5
        seed=20250823

      Adjacency (buyer-seller connectivity):
        adj=random:0.5        # Erdos–Rényi bipartite with density=0.5 (default)
        adj=clustered         # uses clustered_membership(I,J)
        adj=full              # all buyers can bid to all sellers

      Clock / asynchrony:
        clock=chaos           # disable generation-guard (allow stale posts)
        clock=generational    # enable generation-guard (default)

      Policies:
        endpoint=lower|upper|qjc|utility_max
        tie=open|qjc_count|qjc_qty
        split=subset_uniform|min_cost|uniform_all|proportional
        budget=reject|scale_uniform|scale_consistent

      Shortcut:
        utility_max           # endpoint=utility_max, tie=qjc_qty, split=min_cost, budget=scale_consistent

    Examples:
      setup_experiment("I=10,J=4,Q=[50,50,40,60],adj=random:0.6,utility_max,seed=7")
      setup_experiment("endpoint=qjc,tie=qjc_qty,split=min_cost,budget=scale_consistent,clock=generational")
    """
    # ---- defaults ----
    I, J = 8, 3
    Q_max_spec: float | np.ndarray = np.array([60, 40, 50], dtype=float)
    reserve_spec: float | np.ndarray | None = None
    epsilon = 2.5
    seed = 20250823
    density = 0.5
    adj_mode = "random"      # 'random' | 'clustered' | 'full'
    endpoint_policy = "qjc"  # 'lower' | 'upper' | 'qjc' | 'utility_max'
    tie_policy = "qjc_qty"   # 'open' | 'qjc_count' | 'qjc_qty'
    split_rule = "subset_uniform"  # 'subset_uniform' | 'min_cost'
    budget_policy = "reject"       # 'reject' | 'scale_uniform' | 'scale_consistent'
    clock = "chaos"          # generational guard | allow stale posts

    def _parse_list(val: str) -> np.ndarray:
        s = val.strip()
        if s.startswith("[") and s.endswith("]"): s = s[1:-1]
        parts = [p.strip() for p in s.split(",") if p.strip() != ""]
        return np.asarray([float(p) for p in parts], dtype=float)

    tokens = []
    if experiment:
        raw = [t.strip() for t in experiment.split(",")]
        buf, bracket = [], 0
        for t in raw:
            bracket += t.count("[") - t.count("]")
            buf.append(t)
            if bracket == 0:
                tokens.append(",".join(buf).strip())
                buf = []
        if buf: tokens.append(",".join(buf).strip())

    for tok in tokens:
        if not tok: continue
        if "=" not in tok:
            if tok.lower() == "utility_max":
                endpoint_policy = "utility_max"
                tie_policy = "qjc_qty"
                split_rule = "min_cost"
                budget_policy = "scale_consistent"
            continue
        key, val = [s.strip() for s in tok.split("=", 1)]
        kl = key.lower()
        if kl == "i": I = int(val)
        elif kl == "j": J = int(val)
        elif kl == "q":
            try:
                arr = _parse_list(val); Q_max_spec = arr; J = int(arr.size)
            except Exception:
                Q_max_spec = float(val)
        elif kl == "reserve":
            try:
                arr = _parse_list(val); reserve_spec = arr; J = int(arr.size)
            except Exception:
                reserve_spec = float(val)
        elif kl == "epsilon": epsilon = float(val)
        elif kl == "seed": seed = int(val)
        elif kl == "density": density = float(val)
        elif kl == "adj":
            v = val.lower()
            if v.startswith("random"):
                adj_mode = "random"
                if ":" in v:
                    try: density = float(v.split(":", 1)[1])
                    except Exception: pass
            elif v in ("clustered", "cluster", "clusters"):
                adj_mode = "clustered"
            elif v in ("full", "complete", "all"):
                adj_mode = "full"
        elif kl == "clock":
            v = val.lower()
            clock = (v in ("chaos", "stale_ok", "asynchronous"))
        elif kl == "endpoint": endpoint_policy = val.lower()
        elif kl == "tie":      tie_policy = val.lower()
        elif kl == "split":    split_rule = val.lower()
        elif kl == "budget":   budget_policy = val.lower()

    # finalize Q_max/reserve
    if isinstance(Q_max_spec, np.ndarray):
        Q_max = Q_max_spec.astype(float); J = int(Q_max.size)
    else:
        Q_max = np.full(J, float(Q_max_spec), dtype=float)

    if reserve_spec is None:
        reserve = np.zeros(J, dtype=float)
    elif isinstance(reserve_spec, np.ndarray):
        reserve = reserve_spec.astype(float)
        if reserve.size != J:
            reserve = np.full(J, float(reserve.ravel()[0]), dtype=float) if reserve.size == 1 else reserve[:J]
    else:
        reserve = np.full(J, float(reserve_spec), dtype=float)

    # adjacency
    if adj_mode == "full":
        adj = np.ones((I, J), dtype=bool)
    elif adj_mode in ("clustered", "cluster", "clusters"):
        adj = clustered_membership(I, J)
    else:
        adj = random_bipartite(I, J, density=density)

    # market
    M = make_market_multi(I, J, Q_max=Q_max, epsilon=epsilon, reserve=reserve, seed=seed, adj=adj)

    # policies & clock
    M["endpoint_policy"] = endpoint_policy
    M["tie_policy"]      = tie_policy
    M["split_rule"]      = split_rule
    M["budget_policy"]   = budget_policy
    M["clock"]           = clock

    # metadata for reproducibility
    M["seed"] = int(seed)
    M["exp_meta"] = {
        "I": I, "J": J,
        "seed": int(seed),
        "epsilon": float(epsilon),
        "adj_mode": adj_mode,
        "density": float(density) if adj_mode == "random" else None,
        "endpoint_policy": endpoint_policy,
        "tie_policy": tie_policy,
        "split_rule": split_rule,
        "budget_policy": budget_policy,
        "clock": clock,
    }

    # canonical string (store for printing/logging)
    M["experiment_str"] = experiment_to_string(M)
    return M

def print_market_config(M: Dict,
                        round_decimals: int = 3,
                        return_frames: bool = True):
    """
    Pretty-print the current market configuration and basic runtime stats.

    Prints:
      - Header with experiment string (if available), sizes, policies, tolerances, clock
      - Seller table: Q_max, reserve, p_star, posted_q_sum, alloc_sum, demand_ratio, winners/losers, undersubscribed
      - Buyer table: qbar, kappa, budget, degree (adj), posted totals

    Returns (if return_frames):
      (seller_df, buyer_df, meta_dict)
    """
    I, J = int(M["I"]), int(M["J"])
    Q_max   = np.asarray(M["Q_max"], float)
    reserve = np.asarray(M.get("reserve", np.zeros(J)), float)
    p_star  = np.asarray(M.get("p_star", np.zeros(J)), float)
    qbar    = np.asarray(M["qbar"], float)
    kappa   = np.asarray(M["kappa"], float)
    budget  = np.asarray(M["b"], float)
    adj     = M.get("adj", np.ones((I, J), dtype=bool))

    # Totals & runtime knobs
    Q_tot     = float(Q_max.sum())
    Qbar_tot  = float(qbar.sum())
    adj_density = float(adj.mean()) if isinstance(adj, np.ndarray) else 1.0

    # Per-seller posted & allocated
    posted_q_sum = np.asarray(M["bid_q"].sum(axis=0), float)

    # Allocations (current bids)
    alloc_sum = np.zeros(J, dtype=float)
    winners   = np.zeros(J, dtype=int)
    losers    = np.zeros(J, dtype=int)
    market_cost = 0.0
    market_val  = 0.0
    for i in range(I):
        a_vec, a_tot = a_row(i, M)
        alloc_sum += a_vec
        winners   += (a_vec > 0.0).astype(int)
        losers    += ((M["bid_q"][i, :] > 0.0) & (a_vec <= 1e-15)).astype(int)
        c_i = cost_row(i, a_vec, M)
        v_i = theta_i(i, float(a_tot), M)
        market_cost += c_i
        market_val  += v_i
    demand_ratio = np.divide(posted_q_sum, Q_max, out=np.zeros_like(posted_q_sum), where=Q_max > 0)
    market_alloc = float(alloc_sum.sum())
    market_util  = float(market_val - market_cost)

    # Per-seller table
    seller_df = pd.DataFrame({
        "Seller": np.arange(J, dtype=int),
        "Q_max": Q_max,
        "reserve": reserve,
        "p_star": p_star,
        "posted_q_sum": posted_q_sum,
        "alloc_sum": alloc_sum,
        "demand_ratio": demand_ratio,
        "winners": winners,
        "losers": losers,
        "undersubscribed": alloc_sum + 1e-12 < Q_max,
    })

    # Per-buyer table
    buyer_df = pd.DataFrame({
        "Buyer": np.arange(I, dtype=int),
        "qbar": qbar,
        "kappa": kappa,
        "budget": budget,
        "deg": adj.sum(axis=1) if isinstance(adj, np.ndarray) else np.full(I, J),
        "posted_q_sum": np.asarray(M["bid_q"].sum(axis=1), float),
        "posted_p_max": np.asarray(M["bid_p"].max(axis=1), float),
    })

    # Meta (runtime config)
    meta = {
        "t": float(M.get("t", 0.0)),
        "I": I, "J": J,
        "Q_tot": Q_tot,
        "Qbar_tot": Qbar_tot,
        "adj_density": adj_density,
        "epsilon": float(M.get("epsilon", 0.0)),
        "jitter": float(M.get("jitter", 0.0)),
        "grid_K": int(M.get("grid_K", 0)),
        "tol": float(M.get("tol", 0.0)),
        "seed": int(M.get("seed", 0)),
        "chaos": bool(M.get("chaos", False)),
        "endpoint_policy": M.get("endpoint_policy"),
        "tie_policy": M.get("tie_policy"),
        "split_rule": M.get("split_rule"),
        "budget_policy": M.get("budget_policy"),
        "experiment": M.get("experiment_str", None),
        "market_alloc": market_alloc,
        "market_value": float(market_val),
        "market_cost": float(market_cost),
        "market_utility": float(market_util),
    }

    # ---- pretty print ----
    def _round_df(df: pd.DataFrame, d: int) -> pd.DataFrame:
        out = df.copy()
        num = out.select_dtypes(include=[np.number]).columns
        if len(num): out[num] = out[num].round(d)
        return out

    print("\n=== Market Configuration ===")
    #if meta["experiment"]:
        #print(f"experiment: {meta['experiment']}")
    print(f"I={I}, J={J}, Q_tot={Q_tot:.3f}, Qbar_tot={Qbar_tot:.3f}, adj_density={adj_density:.3f}")
    print(f"epsilon={meta['epsilon']}, jitter={meta['jitter']}, grid_K={meta['grid_K']}, tol={meta['tol']}")
    print(f"policies: endpoint={meta['endpoint_policy']}, tie={meta['tie_policy']}, "
          f"split={meta['split_rule']}, budget={meta['budget_policy']}, chaos={meta['chaos']}")
    print(f"runtime: t={meta['t']:.3f}, seed={meta['seed']}")
    print(f"market totals: alloc={market_alloc:.3f}, value={market_val:.3f}, "
          f"cost={market_cost:.3f}, utility={market_util:.3f}")

    print("\n-- Sellers --")
    with pd.option_context("display.max_rows", None, "display.width", 200):
        print(_round_df(seller_df, round_decimals).to_string(index=False))

    print("\n-- Buyers --")
    # add quick stats for buyers
    q_stats = (qbar.min(), qbar.mean(), qbar.max())
    k_stats = (kappa.min(), kappa.mean(), kappa.max())
    b_stats = (budget.min(), budget.mean(), budget.max())
    print(f"qbar stats (min/mean/max): {q_stats[0]:.3f}/{q_stats[1]:.3f}/{q_stats[2]:.3f}")
    print(f"kappa stats (min/mean/max): {k_stats[0]:.3f}/{k_stats[1]:.3f}/{k_stats[2]:.3f}")
    print(f"budget stats (min/mean/max): {b_stats[0]:.3f}/{b_stats[1]:.3f}/{b_stats[2]:.3f}")
    with pd.option_context("display.max_rows", None, "display.width", 200):
        print(_round_df(buyer_df, round_decimals).to_string(index=False))

    return (seller_df, buyer_df, meta) if return_frames else None

# ------------------------------
# Initialization helpers
# ------------------------------

def randomize_bids(M: Dict):
    rng = M["rng"]
    I, J = int(M["I"]), int(M["J"])
    include_prob = 0.4
    for i in range(I):
        z_total = float(rng.uniform(0.0, float(M["qbar"][i])))
        if J == 0 or z_total <= 0.0:
            continue
        # restrict candidate sellers to those connected to i
        cand = np.nonzero(M["adj"][i, :])[0]
        if cand.size == 0:
            continue
        mask = np.zeros(J, dtype=bool)
        # choose subset from cand
        chosen = cand[rng.random(cand.size) < include_prob]
        if chosen.size == 0:
            chosen = np.array([rng.choice(cand)])
        mask[chosen] = True
        idxs = np.nonzero(mask)[0]
        k = len(idxs)
        weights = rng.dirichlet(np.ones(k)) if k > 1 else np.array([1.0])
        shares = z_total * weights
        for t, j in enumerate(idxs):
            M["bid_q"][i, j] = float(shares[t])
        p_uniform = theta_i_prime(i, z_total, M)
        for j in idxs:
            M["bid_p"][i, j] = float(p_uniform)
    for j in range(J):
        _update_p_star_j(M, int(j))

def clustered_membership(I: int, J: int) -> np.array:
    adj = np.zeros((I, J), dtype=bool)
    adj[0:4, 0:2] = True   # group A
    adj[4:8, 1:3] = True   # group B

def random_bipartite(I:int, J: int, density: float) -> np.array:
    return (np.random.default_rng(42).random((I, J)) < density)

def ensure_lamport(M: Dict):
    """Ensure Lamport state exists; no-ops if already present."""
    I, J = int(M["I"]), int(M["J"])
    if "lc_buyer" not in M or M["lc_buyer"] is None:
        M["lc_buyer"] = np.zeros(I, dtype=np.int64)
    if "lc_cell" not in M or M["lc_cell"] is None:
        M["lc_cell"] = np.zeros((I, J), dtype=np.int64)
    # basic metrics buckets
    M.setdefault("stale_drops", 0)
    M.setdefault("retro_applies", 0)
    M.setdefault("retro_hist", {})
    M.setdefault("retro_mass", 0)

# ------------------------------
# Posted price diagnostic (simple 2nd price among q>0 posters)
# ------------------------------

def _alloc_greedy_j(M: Dict, j: int) -> np.ndarray:
    """Greedy allocation at seller j by descending price (diagnostic only).
    Returns alloc[i] with total <= Q_max[j]."""
    I = M["I"]
    prices = M["bid_p"][:, j]
    qty = M["bid_q"][:, j]
    order = np.argsort(-prices, kind="mergesort") # stable descending
    cap = float(M["Q_max"][j])
    alloc = np.zeros(I, dtype=float)
    for i in order:
        if cap <= 0.0:
            break
        if prices[i] <= 0.0 or qty[i] <= 1e-15:
            continue
        take = min(float(qty[i]), cap)
        alloc[i] = take
        cap -= take
    return alloc

def _update_p_star_j(M: Dict, j: int):
    """Set p_star[j] as the second-highest posted price among buyers who actually get alloc>0 at seller j."""
    alloc = _alloc_greedy_j(M, j)
    winners = (alloc > 0.0)
    if not np.any(winners):
        M["p_star"][j] = float(M["reserve"][j])
        return
    active_prices = M["bid_p"][winners, j]
    if active_prices.size >= 2:
        idx = np.argpartition(-active_prices, 1)[:2]
        M["p_star"][j] = float(np.min(active_prices[idx]))
    else:
        M["p_star"][j] = float(M["reserve"][j])

# ------------------------------
# Reporting helpers
# ------------------------------

def market_snapshot(M: Dict) -> pd.DataFrame:
    I, J = M["I"], M["J"]
    rows = []
    for i in range(I):
        a, atot = a_row(i, M)
        cost = cost_row(i, a, M)
        row = {
            "i": i,
            "a_total": atot,
            "qbar": float(M["qbar"][i]),
            "kappa": float(M["kappa"][i]),
            "u_i": theta_i(i, atot, M) - cost,
            "cost": cost,
        }
        # include a few columns for readability
        for j in range(J):
            row[f"q[{j}]"] = float(M["bid_q"][i, j])
            row[f"p[{j}]"] = float(M["bid_p"][i, j])
        rows.append(row)
    df = pd.DataFrame(rows)
    return df

def market_totals(M: Dict) -> Tuple[float, float, float]:
    I = M["I"]
    total_alloc = 0.0; total_value = 0.0; total_util = 0.0
    for i in range(I):
        a, atot = a_row(i, M)
        cost = cost_row(i, a, M)
        total_alloc += atot
        total_value += theta_i(i, atot, M)
        total_util += theta_i(i, atot, M) - cost
    return total_alloc, total_value, total_util

def market_totals_by_seller(M: Dict) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Note: Valuation/utility attribution is for reporting (it partitions totals across sellers);
    PSP revenue (actual payments) is not returned here—use market_revenue_by_seller().
    """
    J = int(M["J"])
    alloc_j = np.zeros(J, dtype=float)
    value_j = np.zeros(J, dtype=float)
    util_j  = np.zeros(J, dtype=float)

    for i in range(int(M["I"])):
        a_vec, atot = a_row(i, M)  # a_ij = min(q_ij, Q̄_i^j(p_ij))
        # accumulate allocations
        alloc_j += a_vec.astype(float)

        # per-seller PSP costs (revenue) for buyer i
        cost_j = np.array([integral_P_i_j(i, j, float(a_vec[j]), M) for j in range(J)], dtype=float)
        cost_i = float(cost_j.sum())

        v_i = theta_i(i, float(atot), M)
        u_i = v_i - cost_i

        if atot > 0.0:
            weights = (a_vec / float(atot))
            value_j += weights * v_i
            util_j  += weights * u_i
        # else: buyer contributes nothing

    return alloc_j, value_j, util_j

def market_revenue_by_seller(M: Dict) -> np.ndarray:
    """
    PSP revenue collected at each seller: rev_j[j] = sum_i ∫_0^{a_ij} P_i^j(z) dz
    """
    J = int(M["J"])
    rev_j = np.zeros(J, dtype=float)
    for i in range(int(M["I"])):
        a_vec, _ = a_row(i, M)
        for j in range(J):
            rev_j[j] += float(integral_P_i_j(i, j, float(a_vec[j]), M))
    return rev_j

def per_seller_totals(M: Dict) -> pd.DataFrame:
    # Per-seller totals
    alloc_j, value_j, util_j = market_totals_by_seller(M)
    rev_j = market_revenue_by_seller(M)

    per_seller_df = pd.DataFrame({
        "seller": np.arange(M["J"]),
        "alloc":  alloc_j,
        "value":  value_j,
        "util":   util_j,
        "revenue": rev_j,
        "Q_max":  M["Q_max"],
        "p_star": M["p_star"],
    }).round(3)
    return per_seller_df

def chaos_report(M: Dict):
    if not bool(M.get("chaos", False)):
        print("Chaos mode is OFF; no Lamport/retrograde stats collected.")
        return
    ra = int(M.get("retro_applies", 0))
    rm = int(M.get("retro_mass", 0))
    hist = M.get("retro_hist", {})
    total = sum(hist.values()) if hist else 0
    top = sorted(hist.items(), key=lambda kv: (-kv[1], -kv[0]))[:8]
    print("--- Chaos / Lamport stats ---")
    print(f"retrograde applies: {ra}")
    print(f"retrograde mass : {rm}")
    print(f"retro histogram : (depth -> count, top {len(top)}/{total})")
    for d,cnt in top:
        print(f" {int(d)} -> {cnt}")
    print("----------------------------")


# ------------------------------
# Pretty printing
# ------------------------------

# ------- core snapshot (long form) -------

def snapshot_long(M: Dict) -> pd.DataFrame:
    """
    Current market snapshot in long form (one row per buyer-seller cell).
    Columns:
      - interval: current engine time M['t'] (float)
      - Buyer, Seller (ints)
      - q_i (posted), p_i (posted)
      - a_i (allocated at current bids)
      - c_ij (PSP cost contribution at seller j)
      - a_total, v_total, c_total, u_total (per-buyer totals, repeated across that buyer’s rows)
    Notes:
      - v_total = θ_i(a_total)
      - c_total = Σ_j c_ij
      - u_total = v_total - c_total
    """
    I, J = int(M["I"]), int(M["J"])
    rows = []
    dt = float(M.get("t", 0.0))

    for i in range(I):
        # buyer-level totals
        a_vec, a_tot = a_row(i, M)
        c_tot = cost_row(i, a_vec, M)
        v_tot = theta_i(i, float(a_tot), M)
        u_tot = v_tot - c_tot

        for j in range(J):
            q_ij = float(M["bid_q"][i, j])
            p_ij = float(M["bid_p"][i, j])
            a_ij = float(a_vec[j])
            c_ij = float(integral_P_i_j(i, j, a_ij, M))
            rows.append({
                "interval": dt,
                "Buyer": i,
                "Seller": j,
                "q_i": q_ij,
                "p_i": p_ij,
                "a_i": a_ij,
                "c_ij": c_ij,
                "a_total": float(a_tot),
                "v_total": float(v_tot),
                "c_total": float(c_tot),
                "u_total": float(u_tot),
            })

    return pd.DataFrame(rows)


# ------- pretty wide print (per-seller blocks) -------

def _round_df(df: pd.DataFrame, decimals: int) -> pd.DataFrame:
    df2 = df.copy()
    num = df2.select_dtypes(include=[np.number]).columns
    if len(num):
        df2[num] = df2[num].round(decimals)
    return df2

def print_snapshot_wide(M: Dict,
                        cols: Tuple[str, ...] = ("Buyer", "q_i", "p_i", "a_i", "c_ij"),
                        round_decimals: int = 3,
                        include_totals: bool = True) -> pd.DataFrame:
    """
    Pretty "round" print: columns grouped by Seller, rows are Buyers.
    `cols` must be a subset of snapshot_long columns. Default shows per-cell q,p,a,c.
    If include_totals=True, appends a rightmost 'Totals' block with a_total, c_total, u_total.
    Returns the wide DataFrame (also printed).
    """
    df_long = snapshot_long(M)
    if df_long.empty:
        print("(snapshot empty)")
        return df_long

    blocks = []
    buyers = sorted(df_long["Buyer"].unique().tolist())

    # Per-seller blocks
    for j in sorted(df_long["Seller"].unique().tolist()):
        sub = df_long[df_long["Seller"] == j][list(cols)].copy()
        # ensure full buyer list order / fill missing
        sub = sub.set_index("Buyer").reindex(buyers).reset_index()
        # make a nice multiindex header
        sub.columns = pd.MultiIndex.from_product([[f"Seller {j}"], sub.columns])
        blocks.append(sub)

    # Optional totals block (one column group)
    if include_totals:
        tot = (df_long.groupby("Buyer", as_index=False)
                      .agg(a_total=("a_total", "first"),
                           c_total=("c_total", "first"),
                           u_total=("u_total", "first")))
        tot.columns = pd.MultiIndex.from_product([["Totals"], tot.columns])
        blocks.append(tot)

    wide = pd.concat(blocks, axis=1)

    wide = _round_df(wide, round_decimals)
    # Pretty print
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 200):
        print(wide.to_string(index=False))
    return wide


# ------- per-seller summary (aggregates & diagnostics) -------

def print_snapshot_summary(M: Dict,
                           round_decimals: int = 2) -> pd.DataFrame:
    """
    Per-seller aggregates for the current snapshot.
    Columns:
      Seller, a_sum, v_sum, u_sum, c_sum
      sum_q (sum of posted q), Q_max, demand_ratio = sum_q/Q_max
      threshold = p_star[j], reserve, winners, losers, undersubscribed (bool)
    Valuation/utility decomposition:
      v_sum and u_sum are allocated to sellers proportional to each buyer’s allocation share:
        v_share_ij = v_total_i * (a_ij / a_total_i)  (0 if a_total_i==0)
        u_share_ij = u_total_i * (a_ij / a_total_i)  (0 if a_total_i==0)
      So Σ_j v_sum[j] equals the market’s total valuation; same for u_sum.
    """
    I, J = int(M["I"]), int(M["J"])
    df = snapshot_long(M)
    if df.empty:
        print("(snapshot empty)")
        return df

    # Per-buyer shares to distribute valuation & utility to sellers
    df = df.copy()
    share = np.where(df["a_total"] > 0.0, df["a_i"] / df["a_total"], 0.0)
    df["v_share"] = df["v_total"] * share
    df["u_share"] = df["u_total"] * share

    # Winners/losers per seller
    winners = df.groupby("Seller")["a_i"].apply(lambda s: int(np.sum(s.values > 0.0)))
    # A "loser" here: posted q>0 but got a=0 at that seller
    losers = df.groupby("Seller").apply(
        lambda g: int(np.sum((g["q_i"].values > 0.0) & (g["a_i"].values <= 0.0)))
    )

    # Per-seller sums
    agg = df.groupby("Seller", as_index=False).agg(
        a_sum=("a_i", "sum"),
        v_sum=("v_share", "sum"),
        u_sum=("u_share", "sum"),
        c_sum=("c_ij", "sum"),
        sum_q=("q_i", "sum"),
    )

    # Attach diagnostics
    agg["Q_max"] = agg["Seller"].map(lambda j: float(M["Q_max"][int(j)]))
    agg["demand_ratio"] = np.where(agg["Q_max"] > 0.0, agg["sum_q"] / agg["Q_max"], np.nan)
    agg["threshold"] = agg["Seller"].map(lambda j: float(M["p_star"][int(j)]) if "p_star" in M else np.nan)
    agg["reserve"]   = agg["Seller"].map(lambda j: float(M["reserve"][int(j)]) if "reserve" in M else 0.0)
    agg["winners"]   = agg["Seller"].map(winners.to_dict())
    agg["losers"]    = agg["Seller"].map(losers.to_dict())
    agg["undersubscribed"] = agg.apply(lambda r: bool(r["a_sum"] + 1e-12 < r["Q_max"]), axis=1)

    agg = _round_df(agg, round_decimals)

    # Pretty print
    header = f"\n--- Snapshot @ t={float(M.get('t',0.0)):.2f} Per-Seller Summary ---"
    print(header)
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 200):
        print(agg.to_string(index=False))
    return agg

import numpy as np
import pandas as pd
from typing import Dict, Tuple, List, Optional

# ------- core snapshot (long form) -------

def snapshot_long(M: Dict) -> pd.DataFrame:
    """
    Current market snapshot in long form (one row per buyer-seller cell).
    Columns:
      - interval: current engine time M['t'] (float)
      - Buyer, Seller (ints)
      - q_i (posted), p_i (posted)
      - a_i (allocated at current bids)
      - c_ij (PSP cost contribution at seller j)
      - a_total, v_total, c_total, u_total (per-buyer totals, repeated across that buyer’s rows)
    Notes:
      - v_total = θ_i(a_total)
      - c_total = Σ_j c_ij
      - u_total = v_total - c_total
    """
    I, J = int(M["I"]), int(M["J"])
    rows = []
    dt = float(M.get("t", 0.0))

    for i in range(I):
        # buyer-level totals
        a_vec, a_tot = a_row(i, M)
        c_tot = cost_row(i, a_vec, M)
        v_tot = theta_i(i, float(a_tot), M)
        u_tot = v_tot - c_tot

        for j in range(J):
            q_ij = float(M["bid_q"][i, j])
            p_ij = float(M["bid_p"][i, j])
            a_ij = float(a_vec[j])
            c_ij = float(integral_P_i_j(i, j, a_ij, M))
            rows.append({
                "interval": dt,
                "Buyer": i,
                "Seller": j,
                "q_i": q_ij,
                "p_i": p_ij,
                "a_i": a_ij,
                "c_ij": c_ij,
                "a_total": float(a_tot),
                "v_total": float(v_tot),
                "c_total": float(c_tot),
                "u_total": float(u_tot),
            })

    return pd.DataFrame(rows)


# ------- pretty wide print (per-seller blocks) -------

def _round_df(df: pd.DataFrame, decimals: int) -> pd.DataFrame:
    df2 = df.copy()
    num = df2.select_dtypes(include=[np.number]).columns
    if len(num):
        df2[num] = df2[num].round(decimals)
    return df2

def print_snapshot_wide(M: Dict,
                        cols: Tuple[str, ...] = ("Buyer", "q_i", "p_i", "a_i", "c_ij"),
                        round_decimals: int = 3,
                        include_totals: bool = True) -> pd.DataFrame:
    """
    Pretty "round" print: columns grouped by Seller, rows are Buyers.
    `cols` must be a subset of snapshot_long columns. Default shows per-cell q,p,a,c.
    If include_totals=True, appends a rightmost 'Totals' block with a_total, c_total, u_total.
    Returns the wide DataFrame (also printed).
    """
    df_long = snapshot_long(M)
    if df_long.empty:
        print("(snapshot empty)")
        return df_long

    blocks = []
    buyers = sorted(df_long["Buyer"].unique().tolist())

    # Per-seller blocks
    for j in sorted(df_long["Seller"].unique().tolist()):
        sub = df_long[df_long["Seller"] == j][list(cols)].copy()
        # ensure full buyer list order / fill missing
        sub = sub.set_index("Buyer").reindex(buyers).reset_index()
        # make a nice multiindex header
        sub.columns = pd.MultiIndex.from_product([[f"Seller {j}"], sub.columns])
        blocks.append(sub)

    # Optional totals block (one column group)
    if include_totals:
        tot = (df_long.groupby("Buyer", as_index=False)
                      .agg(a_total=("a_total", "first"),
                           c_total=("c_total", "first"),
                           u_total=("u_total", "first")))
        tot.columns = pd.MultiIndex.from_product([["Totals"], tot.columns])
        blocks.append(tot)

    wide = pd.concat(blocks, axis=1)

    wide = _round_df(wide, round_decimals)
    # Pretty print
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 200):
        print(wide.to_string(index=False))
    return wide


# ------- per-seller summary (aggregates & diagnostics) -------

def print_snapshot_summary(M: Dict,
                           round_decimals: int = 2) -> pd.DataFrame:
    """
    Per-seller aggregates for the current snapshot.
    Columns:
      Seller, a_sum, v_sum, u_sum, c_sum
      sum_q (sum of posted q), Q_max, demand_ratio = sum_q/Q_max
      threshold = p_star[j], reserve, winners, losers, undersubscribed (bool)
    Valuation/utility decomposition:
      v_sum and u_sum are allocated to sellers proportional to each buyer’s allocation share:
        v_share_ij = v_total_i * (a_ij / a_total_i)  (0 if a_total_i==0)
        u_share_ij = u_total_i * (a_ij / a_total_i)  (0 if a_total_i==0)
      So Σ_j v_sum[j] equals the market’s total valuation; same for u_sum.
    """
    I, J = int(M["I"]), int(M["J"])
    df = snapshot_long(M)
    if df.empty:
        print("(snapshot empty)")
        return df

    # Per-buyer shares to distribute valuation & utility to sellers
    df = df.copy()
    share = np.where(df["a_total"] > 0.0, df["a_i"] / df["a_total"], 0.0)
    df["v_share"] = df["v_total"] * share
    df["u_share"] = df["u_total"] * share

    # Winners/losers per seller
    winners = df.groupby("Seller")["a_i"].apply(lambda s: int(np.sum(s.values > 0.0)))
    # A "loser" here: posted q>0 but got a=0 at that seller
    losers = df.groupby("Seller").apply(
        lambda g: int(np.sum((g["q_i"].values > 0.0) & (g["a_i"].values <= 0.0)))
    )

    # Per-seller sums
    agg = df.groupby("Seller", as_index=False).agg(
        a_sum=("a_i", "sum"),
        v_sum=("v_share", "sum"),
        u_sum=("u_share", "sum"),
        c_sum=("c_ij", "sum"),
        sum_q=("q_i", "sum"),
    )

    # Attach diagnostics
    agg["Q_max"] = agg["Seller"].map(lambda j: float(M["Q_max"][int(j)]))
    agg["demand_ratio"] = np.where(agg["Q_max"] > 0.0, agg["sum_q"] / agg["Q_max"], np.nan)
    agg["threshold"] = agg["Seller"].map(lambda j: float(M["p_star"][int(j)]) if "p_star" in M else np.nan)
    agg["reserve"]   = agg["Seller"].map(lambda j: float(M["reserve"][int(j)]) if "reserve" in M else 0.0)
    agg["winners"]   = agg["Seller"].map(winners.to_dict())
    agg["losers"]    = agg["Seller"].map(losers.to_dict())
    agg["undersubscribed"] = agg.apply(lambda r: bool(r["a_sum"] + 1e-12 < r["Q_max"]), axis=1)

    agg = _round_df(agg, round_decimals)

    # Pretty print
    header = f"\n--- Snapshot @ t={float(M.get('t',0.0)):.2f} Per-Seller Summary ---"
    print(header)
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 200):
        print(agg.to_string(index=False))
    return agg

# ------------------------------
# Valuation and utility surface plots
# ------------------------------

def plot_buyer_diagnostics(M: Dict, i: int, *, show_jbr: bool = True, num_points: int = 200):
    """
    One-stop diagnostic for buyer i:
      - Left: valuation θ_i(z) with current total allocation Z_cur marked.
      - Right: marginal θ'_i(z) with (Z_cur, θ'_i(Z_cur)), seller price lines p[i,j],
               and (optionally) the JBR target point (Z_hat, w_hat) from a dry-run.
      - Prints deviation metrics: max |p[i,j] - θ'_i(Z_cur)| on active sellers, spread of p’s, etc.
    """
    # ----- current snapshot totals -----
    a_vec, Z_cur = a_row(i, M)                              # allocations under current bids
    val_cur = theta_i(i, float(Z_cur), M)
    w_eff   = theta_i_prime(i, float(Z_cur), M)

    # Per-seller PSP cost (for info box)
    J = int(M["J"])
    cost_cur = sum(float(integral_P_i_j(i, j, float(a_vec[j]), M)) for j in range(J))
    util_cur = val_cur - cost_cur

    # Active sellers for this buyer
    active = (a_vec > 1e-12)
    p_row = M["bid_p"][i, :]
    p_active = p_row[active] if np.any(active) else np.array([], dtype=float)
    max_dev = float(np.max(np.abs(p_active - w_eff))) if p_active.size else 0.0
    spread  = float(np.ptp(p_active)) if p_active.size > 1 else 0.0

    # ----- optional JBR target (dry run; not applied) -----
    Z_hat, w_hat = None, None
    if show_jbr:
        q_hat, p_hat, feasible, u_hat = joint_best_response_plan(i, M)
        if feasible:
            Z_hat = float(np.sum(q_hat))
            w_hat = float(p_hat[0]) if len(p_hat) else 0.0

    # ----- curves -----
    zmax = float(M["qbar"][i])
    Zs = np.linspace(0.0, max(zmax, 1e-12), num_points)
    vals  = np.array([theta_i(i, float(z), M)       for z in Zs], dtype=float)
    mvals = np.array([theta_i_prime(i, float(z), M) for z in Zs], dtype=float)

    # ----- plotting -----
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axV, axM = axes

    # Valuation
    axV.plot(Zs, vals, label=r"$\theta_i(z)$")
    axV.axvline(float(Z_cur), linestyle="--", linewidth=0.8)
    axV.scatter([Z_cur], [val_cur], zorder=5, label="current")
    if show_jbr and (Z_hat is not None):
        axV.axvline(Z_hat, linestyle=":", linewidth=0.8)
        axV.scatter([Z_hat], [theta_i(i, Z_hat, M)], marker="x", zorder=6, label="JBR target")

    axV.set_xlabel("Total quantity z")
    axV.set_ylabel(r"$\theta_i(z)$")
    axV.set_title(f"Buyer {i}: valuation")
    axV.legend(loc="best")
    axV.set_xlim(0.0, zmax * 1.1 if zmax > 0 else 1.0)

    # Marginal
    axM.plot(Zs, mvals, label=r"$\theta'_i(z)$")
    axM.scatter([Z_cur], [w_eff], zorder=5, label="current (z, θ')")
    if show_jbr and (w_hat is not None):
        axM.scatter([Z_hat], [w_hat], marker="x", zorder=6, label="JBR (ẑ, ŵ)")

    # Seller price lines for this buyer
    for j in range(J):
        pj = float(p_row[j])
        axM.axhline(pj, linestyle="--", linewidth=0.8, alpha=0.5)
        # annotate on the right margin
        axM.text(zmax * 1.01 if zmax > 0 else 0.01, pj, f" p[{j}]={pj:.2f}",
                 va="center", fontsize=8)

    axM.set_xlabel("Total quantity z")
    axM.set_ylabel("Marginal value / Price")
    axM.set_title(f"Buyer {i}: marginal vs. prices")
    axM.legend(loc="best")
    ylim_max = max(
        (mvals.max() if mvals.size else 1.0),
        (float(np.max(p_row)) if J > 0 else 0.0)
    ) * 1.1
    if ylim_max <= 0: ylim_max = 1.0
    axM.set_xlim(0.0, zmax * 1.1 if zmax > 0 else 1.0)
    axM.set_ylim(0.0, ylim_max)

    # Info box (right)
    txt = [
        f"Z_cur={Z_cur:.3f}",
        f"θ'(Z_cur)={w_eff:.3f}",
        f"val={val_cur:.3f}, cost={cost_cur:.3f}, util={util_cur:.3f}",
        f"active sellers={int(np.sum(active))}",
        f"max|p-θ'| (active)={max_dev:.3e}",
        f"spread(p_active)={spread:.3e}",
    ]
    if show_jbr and (Z_hat is not None):
        txt += [f"JBR: ẑ={Z_hat:.3f}, ŵ={w_hat:.3f}"]
    axM.text(0.98, 0.02, "\n".join(txt), ha="right", va="bottom",
             transform=axM.transAxes, fontsize=9,
             bbox=dict(boxstyle="round,pad=0.3", alpha=0.1))

    fig.suptitle(f"Buyer {i}: valuation & marginal diagnostics", y=1.02)
    fig.tight_layout()
    plt.show()

def plot_utility_surface(M: Dict, i: int, j: int, q_steps: int = 50, p_steps: int = 50):
    """Plot u_i under PSP cost while varying (q,p) at seller j.
    """
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D # noqa: F401 (needed for 3D projection)

    zs = np.linspace(0.0, float(M["Q_max"][j]), q_steps)
    w_max = theta_i_prime(i, 0.0, M)
    ws = np.linspace(0.0, max(w_max, 1e-12), p_steps)
    Z, W = np.meshgrid(zs, ws)
    U = np.zeros_like(Z)

    # snapshot original cell and compute
    q_old = float(M["bid_q"][i, j]); p_old = float(M["bid_p"][i, j])
    for r in range(p_steps):
      for c in range(q_steps):
        M["bid_q"][i, j] = float(Z[r, c])
        M["bid_p"][i, j] = float(W[r, c])
        U[r, c] = u_i_current(i, M)
    M["bid_q"][i, j] = q_old
    M["bid_p"][i, j] = p_old

    print(f"Utility range (buyer {i}, seller {j}): {U.min():.6f} .. {U.max():.6f}")
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(Z, W, U, rstride=1, cstride=1)
    ax.set_xlabel('quantity z to seller j')
    ax.set_ylabel('price w to seller j')
    ax.set_zlabel('u_i (PSP)')
    ax.set_title(f'Utility Surface (buyer {i}, seller {j})')
    plt.tight_layout(); plt.show()

# ------------------------------
# Connectivity plots
# ------------------------------

def plot_connectivity(M: Dict, *, title: Optional[str] = None, show_labels: bool = True):
    """
    Visualize market connectivity M["adj"] (I x J boolean) with:
      - Left: matrix scatter (buyer index vs seller index)
      - Right: bipartite layout (buyers at y=0, sellers at y=1, edges as line segments)
    """
    if "adj" not in M:
        raise ValueError("M['adj'] not found. Pass an (I x J) boolean adjacency matrix in make_market_multi(...).")

    adj = np.asarray(M["adj"], dtype=bool)
    I, J = adj.shape
    ii, jj = np.nonzero(adj)  # edge endpoints (buyer i, seller j)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axL, axR = axes

    # --- Left: matrix scatter view ---
    axL.scatter(jj, ii, s=25, marker='s')
    axL.set_xlabel("seller j")
    axL.set_ylabel("buyer i")
    axL.set_xlim(-0.5, J - 0.5)
    axL.set_ylim(I - 0.5, -0.5)  # buyer 0 at top
    axL.set_xticks(range(J))
    axL.set_yticks(range(I))
    axL.set_title("Adjacency (matrix scatter)")
    axL.grid(True, linestyle=":", linewidth=0.5, alpha=0.4)

    # --- Right: bipartite scatter view ---
    # buyers at y=0, sellers at y=1
    x_b = np.arange(I); y_b = np.zeros(I)
    x_s = np.arange(J); y_s = np.ones(J)

    axR.scatter(x_b, y_b, s=40, label="buyers")
    axR.scatter(x_s, y_s, s=60, marker="^", label="sellers")

    # Build edge segments efficiently
    if ii.size:
        segments = [((x_b[i], 0.0), (x_s[j], 1.0)) for i, j in zip(ii, jj)]
        lc = LineCollection(segments, linewidths=0.8, alpha=0.35)
        axR.add_collection(lc)

    if show_labels:
        for i in range(I):
            axR.text(x_b[i], -0.06, f"b{i}", ha="center", va="top", fontsize=8)
        for j in range(J):
            axR.text(x_s[j], 1.06, f"s{j}", ha="center", va="bottom", fontsize=8)

    axR.set_xlim(-0.5, max(I, J) - 0.5)
    axR.set_ylim(-0.4, 1.4)
    axR.set_yticks([0, 1]); axR.set_yticklabels(["buyers", "sellers"])
    axR.set_xticks([])
    axR.set_title("Adjacency (bipartite scatter)")
    axR.legend(loc="upper right", fontsize=8, frameon=False)

    if title:
        fig.suptitle(title, y=1.02)
    fig.tight_layout()
    plt.show()
